In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib
import cv2
import numpy as np
import os



# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
# instantiating the model in the strategy scope creates the model on the TPU

AUTO = tf.data.experimental.AUTOTUNE

IMAGE_SIZE = [331, 331]

batch_size = 16 * tpu_strategy.num_replicas_in_sync

data_dir = "../input/capillaroscopy/Images/*.jpg"
'''
validation_split = 0.19
filenames = tf.io.gfile.glob(data_dir)
split = len(filenames) - int(len(filenames) * validation_split)
train_fns = filenames[:split]
validation_fns = filenames[split:]

print(train_fns)
'''
data_path = pathlib.Path(data_dir)

print('Database Directory')
print(data_path)

#image_count = len(list(data_path.glob('*.jpg')))
#print(image_count)

retinal_dataset = sorted(list(data_path.glob('*')))

batch_size = 8


#Preprocessing
training_data = []
def read_image(path):
    for img in path:
        im_path = str(img)
        img_array = cv2.imread(im_path)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        img_array = cv2.resize(img_array, dsize=(576, 544), interpolation=cv2.INTER_AREA)
        training_data.append(img_array)
        if len(training_data)>400:
            break

read_image(retinal_dataset) 


X = np.array(training_data)
print(X.dtype)
X = X.astype('float32') / 255.0
print(X.dtype)


IMG_SHAPE = X.shape[1:]
from sklearn.model_selection import train_test_split

train_imgs, test_imgs = train_test_split(X, test_size =0.2)


noise_factor = 0.1
x_train_noisy = train_imgs + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train_imgs.shape) 
x_test_noisy = test_imgs + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=test_imgs.shape) 

train_noisy = np.clip(x_train_noisy, 0.0, 1.0)
test_noisy = np.clip(x_test_noisy, 0.0, 1.0)
# plot image example from training images
plt.imshow(train_imgs[1])
plt.show()

plt.imshow(train_noisy[1])
plt.show()
'''
def add_noise(img):
        VARIABILITY = 50
        deviation = VARIABILITY*random.random()
        noise = np.random.normal(0, deviation, img.shape)
        img += noise
        np.clip(img, 0.0, 255.0)
        return img
#Data Generator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=40,
                                                               width_shift_range=0.2,
                                                               height_shift_range=0.2,
                                                               zoom_range=0.4,
                                                               validation_split=0.2, 
                                                               preprocessing_function=add_noise)
train_generator = train_datagen.flow_from_directory(data_dir, batch_size=8, color_mode='rgb', target_size=(576, 544), class_mode=None, subset='training')
validation_generator = train_datagen.flow_from_directory(data_dir, batch_size=8, color_mode='rgb', target_size=(576, 544), class_mode=None, subset='validation')
#image_count = len(list(data_path.glob('*.jpg')))

    history = auto_encoder.fit(x=train_generator, steps_per_epoch=train_generator.samples,
                              validation_data=validation_generator,
                              validation_steps=validation_generator.samples,
                              epochs=10)
'''
with tpu_strategy.scope():
    input_img = tf.keras.layers.Input(IMG_SHAPE)
    l1 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(input_img)
    l2 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l1)
    r1 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l2)
    r2 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(r1)
    l3 = tf.keras.layers.MaxPool2D(padding='same')(r2)

    l4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l3)
    l5 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l4)
    r4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l5)
    r5 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(r4)
    l6 = tf.keras.layers.MaxPool2D(padding='same')(r5)

    la = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l6)
    lb = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(la)
    ra = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(lb)
    rb = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(ra)
    lc = tf.keras.layers.MaxPool2D(padding='same')(rb)

    
    l7 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(lc)

    
    ld = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, padding='same', strides=2)(l7)
    le = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(ld)
    lf = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(le)
    rd = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(lf)
    re = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(rd)
    lg = tf.keras.layers.add([re, rb])
    
    #l8 = tf.keras.layers.UpSampling2D()(l7)
    l8 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, padding='same', strides=2)(lg)
    l9 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l8)
    l10 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l9)
    r9 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l10)
    r10 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(r9)
    l11 = tf.keras.layers.add([r10, r5])
    #l11 = r10
    l12 = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=3, padding='same', strides=2)(l11)
    l13 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l12)
    l14 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l13)
    r14 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l14)
    r15 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(r14)
    l15 = tf.keras.layers.add([r15, r2])
    #l15 = r15
    c1 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(l15)
    c2 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', use_bias=True, activity_regularizer=tf.keras.regularizers.l1(10e-10))(c1)
    decoded_image = tf.keras.layers.Conv2D(3, (3, 3), padding='same', kernel_initializer=tf.initializers.GlorotUniform, activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(c2)

    from keras.layers import Dense, Flatten, Reshape, Input, InputLayer
    from keras.models import Sequential, Model

    # training process params
    auto_encoder = Model(inputs=(input_img), outputs=decoded_image)
   # auto_encoder.load_weights(retinal_model)
    auto_encoder.compile(optimizer='adamax', loss='mse')

    print(auto_encoder.summary())

    history = auto_encoder.fit(x=train_noisy, y=train_imgs, epochs=50,
                validation_data=[test_noisy, test_imgs])


import numpy as np
predictions = np.clip(auto_encoder.predict(test_noisy), 0.0, 1.0)
plt.imshow(test_noisy[10])
plt.show()
plt.imshow(predictions[10])
plt.show()
for i in range(20, 35):
    arr = np.uint8(test_noisy[i]*255)
    test = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
    pred = cv2.cvtColor(predictions[i], cv2.COLOR_RGB2BGR)
    cv2.imwrite("noisy_"+str(i)+".png", test)
    cv2.imwrite("denoised_"+str(i)+".png", 255*pred)

#import h5py
#auto_encoder.save_weights('model_save_path.h5')